In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread("/Users/AdamHarris/Desktop/detected_octagons.jpg")
if img is None:
    raise FileNotFoundError("Image not found. Make sure 'maze.jpg' is in the working directory.")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply a binary threshold
# You might need to experiment with the threshold value (e.g., 100, 120, 150)
# or try cv2.THRESH_OTSU for an adaptive approach.
_, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

# Optional: Morphological operations to reduce noise and solidify shapes
# For example, we can use a closing operation (dilate then erode) 
# to fill small holes within the platforms.
kernel = np.ones((3,3), np.uint8)
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

# Find contours of the platforms
# RETR_EXTERNAL will only find external contours which is likely sufficient 
# if the platforms are well-separated.
# CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments 
# and leaves only their end points.
contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out small contours that are not likely to be platforms
# You may adjust this area threshold depending on your image scale
min_area = 5000
platform_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

# Draw the contours on a copy of the original image for visualization
img_contours = img.copy()
cv2.drawContours(img_contours, platform_contours, -1, (0,0,255), 2)

# Create a binary mask for each platform
masks = []
for i, cnt in enumerate(platform_contours):
    # Create a blank mask
    mask = np.zeros_like(gray)
    # Fill the contour on the mask
    cv2.drawContours(mask, [cnt], -1, 255, -1)
    masks.append(mask)

    # Optionally, save each mask as an image
    # Each mask highlights one platform
    cv2.imwrite(f"/Users/AdamHarris/Desktop/platform_mask_{i}.png", mask)

# If you want a single mask highlighting all platforms:
all_platforms_mask = np.zeros_like(gray)
for cnt in platform_contours:
    cv2.drawContours(all_platforms_mask, [cnt], -1, 255, -1)
cv2.imwrite("/Users/AdamHarris/Desktop/all_platforms_mask.png", all_platforms_mask)

# Display results (press any key to close windows)
cv2.imshow("Original Image", img)
cv2.imshow("Thresholded", closed)
cv2.imshow("Contours", img_contours)
cv2.imshow("All Platforms Mask", all_platforms_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import os
os.chdir('/Users/AdamHarris/Desktop/')
# Load the already segmented binary mask (where platforms are white)
# Assume this is a single-channel binary image (0 or 255)
binary = cv2.imread("all_platforms_mask.png", cv2.IMREAD_GRAYSCALE)
if binary is None:
    raise FileNotFoundError("Could not find segmented_mask.png")

# A function to compute circularity given a contour
def compute_circularity(cnt):
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    if perimeter == 0:
        return 0
    circularity = (4 * np.pi * area) / (perimeter * perimeter)
    return circularity

# Find initial contours
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Parameters you may need to adjust:
min_area = 5000       # minimum area a valid platform should have
circularity_threshold = 0.6  # if contour's circularity < 0.6, consider it might be multiple platforms joined

# Expected grid: 3x3
# Let's say we know approximate coordinates for each platform (center positions in the image)
# You should measure these from your image (example placeholders here):
image_height, image_width = binary.shape
# Hypothetical approximate grid coordinates (cx, cy) for each platform:
# You need to determine these coordinates based on your setup.
# For demonstration, let's assume they're spaced evenly. 
grid_x = np.linspace(image_width*0.2, image_width*0.8, 3)   # adjust as needed
grid_y = np.linspace(image_height*0.2, image_height*0.8, 3) # adjust as needed
expected_centers = [(int(x), int(y)) for y in grid_y for x in grid_x]

# Let’s store final masks of individual platforms
final_masks = []

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area < min_area:
        # Too small to be a platform; ignore
        continue

    # Check circularity
    circ = compute_circularity(cnt)
    
    # Extract the ROI of this contour to apply watershed if needed
    x, y, w, h = cv2.boundingRect(cnt)
    contour_mask = np.zeros((h, w), dtype=np.uint8)
    cv2.drawContours(contour_mask, [cnt - [x,y]], -1, 255, -1)  # draw contour localized in ROI

    if circ < circularity_threshold:
        # This suggests multiple platforms stuck together.
        # Use distance transform and watershed to separate.
        
        # Distance transform
        dist = cv2.distanceTransform(contour_mask, cv2.DIST_L2, 5)
        # Normalize for visualization/thresholding
        dist_norm = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Threshold to find peaks
        ret, peaks = cv2.threshold(dist_norm, 200, 255, cv2.THRESH_BINARY)
        # Find markers
        markers_contours, _ = cv2.findContours(peaks, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        markers = np.zeros_like(dist, dtype=np.int32)
        for i, mc in enumerate(markers_contours, 1):
            cv2.drawContours(markers, [mc], -1, i, -1)
        
        # Apply watershed on the ROI
        # Convert ROI to BGR for watershed
        roi_bgr = cv2.cvtColor(contour_mask, cv2.COLOR_GRAY2BGR)
        cv2.watershed(roi_bgr, markers)
        
        # Each marker > 1 corresponds to a separated object
        for m_id in range(1, len(markers_contours)+1):
            mask_temp = np.zeros_like(contour_mask)
            mask_temp[markers == m_id] = 255
            # Add back offset
            full_mask = np.zeros_like(binary)
            full_mask[y:y+h, x:x+w] = mask_temp
            final_masks.append(full_mask)
    else:
        # Contour is already fairly circular; likely a single platform
        # Just add this as a final mask
        single_mask = np.zeros_like(binary)
        cv2.drawContours(single_mask, [cnt], -1, 255, -1)
        final_masks.append(single_mask)

# At this point, we have a list of individual platform masks, but we might have missed or occluded platforms.

# Check if we have 9 platforms:
if len(final_masks) < 9:
    # We know it should be a 3x3 arrangement.
    # Identify which expected positions are unoccupied.
    # One approach: For each expected center, check if there's a mask covering it.
    final_combined = np.zeros_like(binary)
    for m in final_masks:
        final_combined = cv2.bitwise_or(final_combined, m)

    for (ex_cx, ex_cy) in expected_centers:
        if final_combined[ex_cy, ex_cx] == 0:
            # No platform found at this location
            # Attempt to "force" a platform shape here. For example, 
            # we can place a filled polygon or run a morphological operation 
            # to approximate a platform.
            
            # As a simple heuristic, we can draw a roughly octagonal shape at expected center:
            # Adjust size and shape as needed
            radius = 300  # platform "radius", tune this
            missing_mask = np.zeros_like(binary)
            cv2.circle(missing_mask, (ex_cx, ex_cy), radius, 255, -1)
            
            # Add this forced platform:
            final_masks.append(missing_mask)

# Combine all final masks into a single binary image
all_final = np.zeros_like(binary)
for m in final_masks:
    all_final = cv2.bitwise_or(all_final, m)

# (Optional) Refine shapes by slight morphological operations
# For example, a closing to ensure nice round shapes:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
all_final = cv2.morphologyEx(all_final, cv2.MORPH_CLOSE, kernel, iterations=2)

# Save the final result
cv2.imwrite("corrected_platforms.png", all_final)


True

In [ ]:
import os
import pandas as pd

def count_good_clusters(kilosort_folder):
    """
    Count the number of clusters labeled as 'good' in Kilosort output.
    
    Parameters
    ----------
    kilosort_folder : str
        Path to the folder containing Kilosort output files.
        
    Returns
    -------
    int
        The number of 'good' clusters.
        
    Raises
    ------
    FileNotFoundError
        If neither cluster_info.tsv nor cluster_group.tsv can be found.
    ValueError
        If the expected 'group' or 'KSLabel' column is not present in the file.
    """

    # Potential Kilosort/Phy output files that contain cluster quality info
    cluster_info_path = os.path.join(kilosort_folder, 'cluster_info.tsv')
    cluster_group_path = os.path.join(kilosort_folder, 'cluster_group.tsv')
    
    # Attempt to load cluster_info.tsv first
    if os.path.isfile(cluster_info_path):
        df = pd.read_csv(cluster_info_path, sep='\t')
        # The 'group' column usually specifies cluster quality in newer versions of phy
        # In older versions, 'KSLabel' is sometimes used to denote the same info.
        if 'group' in df.columns:
            good_clusters = df[df['group'] == 'good']
        elif 'KSLabel' in df.columns:
            good_clusters = df[df['KSLabel'] == 'good']
        else:
            raise ValueError(f"No 'group' or 'KSLabel' column found in {cluster_info_path}")
        
        return len(good_clusters)
    
    # If cluster_info.tsv not available, try cluster_group.tsv
    elif os.path.isfile(cluster_group_path):
        # cluster_group.tsv has a simpler format, usually two columns:
        # cluster_id <tab> group_label
        df = pd.read_csv(cluster_group_path, sep='\t', header=None, names=['cluster_id', 'group'])
        good_clusters = df[df['group'] == 'good']
        return len(good_clusters)
    
    else:
        # Neither cluster_info nor cluster_group found
        raise FileNotFoundError("Neither cluster_info.tsv nor cluster_group.tsv found in the provided folder.")

# Example usage:
# num_good = count_good_clusters('/path/to/kilosort/output')
# print(f"Number of good clusters: {num_good}")


In [31]:
import os

def count_good_clusters(kilosort_output_path: str) -> int:
    """
    Count the number of 'good' clusters in a Kilosort output directory.
    
    Parameters
    ----------
    kilosort_output_path : str
        Path to the directory containing Kilosort output files.
        
    Returns
    -------
    int
        The number of clusters labeled as 'good'.
    """
    # Common files to look for. Adjust if you have a different naming scheme.
    # This is the file that often contains cluster IDs and labels.
    ks_label_file = os.path.join(kilosort_output_path, 'cluster_info.tsv')
    
    # If the Kilosort output uses a different file naming scheme, you may need
    # to change this. For instance, if you only have 'cluster_info.tsv', you can
    # search for a column that indicates whether a cluster is 'good' or not.
    
    # if not os.path.exists(ks_label_file):
    #     raise FileNotFoundError(f"No cluster_KSLabel.tsv file found at {ks_label_file}")
    
    good_count = 0
    with open(ks_label_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Typical line format: "cluster_id    label"
            parts = line.split()
            if len(parts) < 2:
                continue
            # The second column is typically the cluster label
            cluster_label = parts[2]
            if cluster_label.lower() == 'good':
                good_count += 1
    
    return good_count

# Example usage:
# path_to_kilosort = '/path/to/kilosort_output'
# n_good = count_good_clusters(path_to_kilosort)
# print(f"Number of good clusters: {n_good}")


In [34]:
folders = ["/Volumes/behrens/adam_harris/Taskspace_abstraction_mEC/Data/cohort7/Sorted/bp01/02042024_03042024_combined_all", 
            "/Volumes/behrens/adam_harris/Taskspace_abstraction_mEC/Data/cohort7/Sorted/bp01/24032024_25032024_combined_all",
            "/Volumes/behrens/adam_harris/Taskspace_abstraction_mEC/Data/cohort7/Sorted/bp01/bp01_21032024_23032024_20241127122138_ALL_202411271709",
            "/Volumes/behrens/adam_harris/Taskspace_abstraction_mEC/Data/cohort7/Sorted/bp01/bp01_28032024_31032024_02042024_20241213161547_ALL_202412140004"]

for i in folders[2:]:
    print(i)
    print(count_good_clusters(i))


/Volumes/behrens/adam_harris/Taskspace_abstraction_mEC/Data/cohort7/Sorted/bp01/bp01_21032024_23032024_20241127122138_ALL_202411271709
0
/Volumes/behrens/adam_harris/Taskspace_abstraction_mEC/Data/cohort7/Sorted/bp01/bp01_28032024_31032024_02042024_20241213161547_ALL_202412140004
0


# Decoding Analysis